In [2]:
from tracemalloc import stop
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
sw=nltk.corpus.stopwords.words("english")

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [4]:
!pip install emoji


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import emoji

In [6]:
df = pd.read_csv("UScomments.csv",error_bad_lines=False)

C:\Users\LENOVO\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3552: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 41589: expected 4 fields, saw 11\nSkipping line 51628: expected 4 fields, saw 7\nSkipping line 114465: expected 4 fields, saw 5\n'
b'Skipping line 142496: expected 4 fields, saw 8\nSkipping line 189732: expected 4 fields, saw 6\nSkipping line 245218: expected 4 fields, saw 7\n'
b'Skipping line 388430: expected 4 fields, saw 5\n'
C:\Users\LENOVO\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
df.sample(10)

,video_id,comment_text,likes,replies
433292,I7TSzpDetTY,Who's the nigger?,1,0
277129,9QZb-659ZfY,Go to Thailand. It's beautiful out there.,0,0
643675,NVvJ3t5eiTU,NinoOoOo likes it RAW,0,0
404395,CwLGro-dFWg,Before 1 million views ?,0,0
577310,tLv3X-aMyWQ,BRs,0,0
431976,-otJ1LJGzcc,NEW Y/T watch > Stephen Paddock Las Vegas Sh...,0,0
428966,3N5WORQxQUo,Korean subs!! Thank you so much😍😍,0,0
342935,GNTtR6ZpUOo,(Dances) THIS MEH JAM,0,0
513212,GmrFu7I-bBE,altidore and bobby wood were also EXTREMELY st...,0,0
69452,F1DxLQbx_bU,Oh my gosh Nick these are amazing! Keep up the...,0,0


In [8]:
# df.isnull().sum()

In [70]:
# !pip install gensim

     -------------------------------------- 24.0/24.0 MB 125.8 kB/s eta 0:00:00
     --------------------------------------- 56.8/56.8 kB 58.4 kB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# df.dropna(inplace=True)

In [10]:
!pip install textblob


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from textblob import TextBlob

In [12]:
polarity = []
for comment in df['comment_text']:
    try:
        polarity.append(TextBlob(comment).sentiment.polarity)
    except:
        polarity.append(0)

In [13]:
df['Polarity'] = polarity

In [14]:
# df.head()

In [15]:
x=[]
for i in df.Polarity:
    if i>=0:
        x.append('Positive')
    else:
        x.append('Negative')
#     else:
#         x.append('Neutral')
        

In [16]:
# x

In [17]:
df.head()

,video_id,comment_text,likes,replies,Polarity
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0,0.0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0,0.0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0,0.0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0,0.0
4,XpVt6Z1Gjjo,trending 😉,3,0,0.0


In [18]:
df['Target'] = x

In [69]:
len(df[df['Target']=='Negative'])
df.iloc[9]

video_id                                              XpVt6Z1Gjjo
comment_text    You and your shit brother may have single hand...
likes                                                           0
replies                                                         0
Polarity                                                -0.135714
Target                                                   Negative
Name: 9, dtype: object

In [68]:
df[df['Target']=='Negative']

,video_id,comment_text,likes,replies,Polarity,Target
9,XpVt6Z1Gjjo,You and your shit brother may have single hand...,0,0,-0.135714,Negative
12,XpVt6Z1Gjjo,Honestly Evan is so annoying. Like its not fun...,0,0,-0.023333,Negative
16,XpVt6Z1Gjjo,Ayyyyoooo Logang what up . This was a hard vl...,1,0,-0.291667,Negative
19,XpVt6Z1Gjjo,Made a lot of people hate youtube - GJ,0,0,-0.800000,Negative
28,XpVt6Z1Gjjo,Evan is a horrible human being he also looks s...,0,0,-0.050000,Negative
...,...,...,...,...,...,...
691297,qRoVlH1OcI4,Fuck trump hes honestly such a fucking idiot,0,5,-0.400000,Negative
691298,qRoVlH1OcI4,Fuck Donald trump he is a piece of shit presid...,0,2,-0.085714,Negative
691316,EoejGgUNmVU,"LP, U look exhausted (i think); I was afraid o...",0,0,-0.066667,Negative
691319,EoejGgUNmVU,What do you do in your past life to end up bet...,0,0,-0.250000,Negative


In [20]:
import string
import re

In [21]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [22]:
new_emoji_df = pd.read_csv(r"ijstable.csv")

In [23]:
new_df = new_emoji_df[1:]

In [24]:
new_df.reset_index().drop(['index'],axis=1)

,Char,Unnamed: 1,Unicode,Occurrences,Position,Neg,Neut,Pos,Sentiment score,Sentiment bar,Unicode name,Unicode block
0,😂,NaN,0x1f602,14622,0.805,0.247,0.285,0.468,0.221,NaN,FACE WITH TEARS OF JOY,Emoticons
1,❤,NaN,0x2764,8050,0.747,0.044,0.166,0.79,0.746,NaN,HEAVY BLACK HEART,Dingbats
2,♥,NaN,0x2665,7144,0.754,0.035,0.272,0.693,0.657,NaN,BLACK HEART SUIT,Miscellaneous Symbols
3,😍,NaN,0x1f60d,6359,0.765,0.052,0.219,0.729,0.678,NaN,SMILING FACE WITH HEART-SHAPED EYES,Emoticons
4,😭,NaN,0x1f62d,5526,0.803,0.436,0.22,0.343,-0.093,NaN,LOUDLY CRYING FACE,Emoticons
...,...,...,...,...,...,...,...,...,...,...,...,...
746,♮,NaN,0x266e,5,0.937,0.125,0.625,0.25,0.125,NaN,MUSIC NATURAL SIGN,Miscellaneous Symbols
747,🅾,NaN,0x1f17e,5,0.977,0.375,0.375,0.25,-0.125,NaN,NEGATIVE SQUARED LATIN CAPITAL LETTER O,Enclosed Alphanumeric Supplement
748,🔄,NaN,0x1f504,5,0.971,0.125,0.75,0.125,0,NaN,ANTICLOCKWISE DOWNWARDS AND UPWARDS OPEN CIRCL...,Miscellaneous Symbols and Pictographs
749,☄,NaN,0x2604,5,0.435,0.125,0.75,0.125,0,NaN,COMET,Miscellaneous Symbols


In [25]:
new_list_emoji_df = [i for i in new_df['Char']]

In [26]:
def devide_emoji_with_text(str):
    new_list_emoji_df = [i for i in new_df['Char']]
#     print(new_list_emoji_df)
    text_list_extract=[]
    emoji_list_extract=[]
#     c=0
    for i in str:
#         print(c)
        if i in new_list_emoji_df:
            emoji_list_extract.append(i)
        else:
            text_list_extract.append(i)
#         c+=1
            
    text = "".join(text_list_extract)
#     print(c)
    return len(emoji_list_extract),emoji_list_extract,text

In [27]:
# no,emoji_list,text=devide_emoji_with_text('The end though made me happy😭🅾😭😍')


In [28]:
# print(no)
# print(emoji_list)
# print(text)

In [36]:
df.isnull().sum()
df.dropna(inplace=True)

In [37]:
def transform_text(text):
#     text1 = convert_emojis_to_word(text)
#     text.encode('utf-8')
    text1=text.lower()
    text1=nltk.word_tokenize(text1)
#     print(text1)
    y=[]
    for i in text1:
#         if i.isalnum():
#         if i.islower() or i.encode('utf-8'):
        if i.islower():
            y.append(i)
    text1=y[:]
    #print(text1)
    y.clear()
    for i in text1:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
#         if i in emoji.distinct_emoji_list(i):
#             y.append(i)
    text1=y[:]
#     print(text1)
#     y.clear()
#     ps=PorterStemmer()
#     for i in text1:
#         y.append(ps.stem(i))
#     print(text1)
    return " ".join(y)

In [30]:
# transform_text(text)

In [38]:
tfidf1=TfidfVectorizer(stop_words=sw,max_features=20)
def transform1(txt1):
    txt2=tfidf1.fit_transform(txt1)
    return txt2.toarray()

In [99]:
df['Target_in_Num'] = df['Target'].map({'Positive':1,'Negative':0})
# df['Target_in_Num']

0         1
1         1
2         1
3         1
4         1
         ..
691395    1
691396    1
691397    1
691398    1
691399    1
Name: Target_in_Num, Length: 691375, dtype: int64

In [98]:
x = transform1(df['comment_text'])
y=df["Target_in_Num"]

0         1
1         1
2         1
3         1
4         1
         ..
691395    1
691396    1
691397    1
691398    1
691399    1
Name: Target_in_Num, Length: 691375, dtype: int64

In [ ]:
# x[0:40]

In [ ]:
# new_str = transform_text("I am Happy")
# tmp = tfidf1.transform([new_str])
# pred = model1.predict(tmp.todense())
# # print(new_str)

In [ ]:
# x_test1[0]

In [100]:
# x_train1,x_test1,y_train1,y_test1=train_test_split(x,y,test_size=0.2,random_state=0)

In [101]:
model1=MultinomialNB()
# model1=RandomForestClassifier()
model1.fit(x_train1,y_train1)

MultinomialNB()

In [108]:
def fun1(str):
#     example_str = str
    new_str = transform_text(str)
    tmp = tfidf1.transform([new_str])
    pred = model1.predict(tmp.todense())
    
    if pred[0]=="Positive":
        return 1
    else:
        return 0

In [109]:
def fun2(num,list_of_extract_emoji):
#     no,emojis = extract_emojis(str)
#     list_emoji_df = [i for i in emoji_df['Emoji_Symbole']]
    sent=0
    cnt=0
    if num==0:
        return 0
    for i in list_of_extract_emoji:
        if i in new_list_emoji_df:
            if new_df.iloc[cnt]['Pos']>new_df.iloc[cnt]['Neg']:
                sent+=float(new_df.iloc[cnt]['Pos'])
            else:
                sent+=float(new_df.iloc[cnt]['Neg'])
        else:
            sent+=0
        cnt+=1
    sent /= num
    
    return sent
 

In [110]:
def devide_emoji_with_text(str):
        new_list_emoji_df = [i for i in new_df['Char']]

        text_list_extract=[]
        emoji_list_extract=[]
        for i in str:
            if i in new_list_emoji_df:
                emoji_list_extract.append(i)
            else:
                text_list_extract.append(i)

        text = "".join(text_list_extract)
        return len(emoji_list_extract),emoji_list_extract,text

In [105]:
def final_result(str):
    
    no,list_emoji,text = devide_emoji_with_text(str)
    print("No of emoji : ",no)
    print("List of emoji : ",list_emoji)
    print("Text : ",text)
    
    print("Result after using Transform Text : ",transform_text(text))
    
    final_sentiment=(fun1(text)+
                     fun2(no,list_emoji)/2)
    a=''
    if (final_sentiment>=0):
        a = 'Positive'
    else:
        a = 'Negative'
    print("Final Sentiment Result : ",a)

In [107]:
final_result('The end though happy😭👍')

No of emoji :  2
List of emoji :  ['😭', '👍']
Text :  The end though happy
Result after using Transform Text :  end though happi
Final Sentiment Result :  Negative


In [46]:
# def final_result_2(str):
    
#     no,list_emoji,text = devide_emoji_with_text(str)
    
#     final_sentiment=(fun1(text)+fun2(no,list_emoji)/2)
# #     a=''
#     if (final_sentiment>=0):
#         return 1
#     else:
#         return 0
    

In [47]:
# str='The end though happy😭👍'
t=df['comment_text']

In [48]:
from tqdm import tqdm

In [59]:
j=0
sum=0
experiment_list=[]
for i in tqdm(range(0,len(t))):
                   
    tmp=t[i]
    j+=1
    prt=final_result_2(tmp)
    experiment_list.append(prt)
    sum+=prt
    
print(sum/j)

  5%|███▌                                                                     | 33873/691375 [12:35<4:04:30, 44.82it/s]


IndexError: single positional indexer is out-of-bounds

In [ ]:
fun1(str)

In [ ]:
fun2(no,emoji_list)

In [ ]:
# new_str = transform_text(str)
# tmp = tfidf1.transform([new_str])
# pred = model1.predict(tmp.todense())
# print(new_str)
# print(pred[0])

In [ ]:
# new_str_trans = transform_text(str)
# new_str_trans

In [ ]:
# ex_tfidf = tfidf1.transform([new_str])

In [ ]:
# pred = model1.predict(ex_tfidf)

In [ ]:
# pred[0]